In [13]:
import mambapy
from mambapy.mamba import Mamba
import numpy as np
import torch

In [5]:
config = mambapy.mamba.MambaConfig(d_model=16, n_layers=2)
mamba=Mamba(config)
mamba

Mamba(
  (layers): ModuleList(
    (0-1): 2 x ResidualBlock(
      (mixer): MambaBlock(
        (in_proj): Linear(in_features=16, out_features=64, bias=False)
        (conv1d): Conv1d(32, 32, kernel_size=(4,), stride=(1,), padding=(3,), groups=32)
        (x_proj): Linear(in_features=32, out_features=33, bias=False)
        (dt_proj): Linear(in_features=1, out_features=32, bias=True)
        (out_proj): Linear(in_features=32, out_features=16, bias=False)
      )
      (norm): RMSNorm()
    )
  )
)

In [7]:
rb = mambapy.mamba.ResidualBlock(config)
rb

ResidualBlock(
  (mixer): MambaBlock(
    (in_proj): Linear(in_features=16, out_features=64, bias=False)
    (conv1d): Conv1d(32, 32, kernel_size=(4,), stride=(1,), padding=(3,), groups=32)
    (x_proj): Linear(in_features=32, out_features=33, bias=False)
    (dt_proj): Linear(in_features=1, out_features=32, bias=True)
    (out_proj): Linear(in_features=32, out_features=16, bias=False)
  )
  (norm): RMSNorm()
)

In [15]:
test_x = torch.as_tensor(np.random.random((10,8,16)), dtype=torch.float32)
rb(test_x)

tensor([[[ 0.0872,  0.5937,  0.8553,  ...,  0.6530,  0.2141,  0.7754],
         [ 0.6444,  0.8852,  0.6280,  ...,  0.0656,  0.1700,  0.0801],
         [ 0.3203,  0.3548,  0.0791,  ...,  0.4334,  0.7192,  0.0883],
         ...,
         [ 0.2129,  0.2162,  0.5279,  ...,  0.7330,  0.3299,  0.8880],
         [ 0.7771,  0.5676,  0.6809,  ...,  0.1005,  0.8802,  0.8964],
         [ 0.0518,  0.3348,  0.9488,  ...,  0.8742,  0.8953,  0.9997]],

        [[ 0.1553,  0.8824,  0.1853,  ...,  0.3263,  0.1944,  0.8926],
         [ 0.2177,  0.8210,  0.8657,  ...,  0.0939,  0.0685,  0.5629],
         [ 0.0500,  0.0127,  0.6679,  ...,  0.0634,  0.1939,  0.7773],
         ...,
         [ 0.5660,  0.3241,  0.6979,  ...,  0.8248,  0.7889,  0.3960],
         [ 0.0957,  0.1477,  0.5040,  ...,  0.5324,  0.2511,  0.9335],
         [ 0.3306,  0.5829,  0.8941,  ...,  0.5937,  0.3152,  0.9648]],

        [[ 0.6643,  0.9192,  0.7827,  ...,  0.5485,  0.1623,  0.1676],
         [ 0.3650,  0.1268,  0.1277,  ...,  0

In [11]:
mamba(test_x)

AttributeError: 'numpy.ndarray' object has no attribute 'pow'